# import libraries (you may add additional imports but you may not have to)

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# get data files

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

# import csv data into dataframes

In [ ]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# Filter dataset

In [ ]:
# Keep users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Keep books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge ratings with book titles to ensure we have a mapping
df = pd.merge(df_ratings, df_books, on='isbn')

# Create a pivot table: rows are book titles, columns are users

In [ ]:
# We fill missing values with 0 (indicating no rating)
book_user_mat = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert the pivot table to a sparse matrix
book_user_sparse = csr_matrix(book_user_mat.values)

# Fit NearestNeighbors model

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

# Reverse mapping of indices to book titles

In [ ]:
book_to_idx = {
    book: i for i, book in enumerate(book_user_mat.index)
}

idx_to_book = {
    i: book for book, i in book_to_idx.items()
}

# function to return recommended books - this will be tested

In [ ]:
def get_recommends(book=""):
    # Check if the book is in our dataset
    if book not in book_to_idx:
        return [book, []]
    
    # Get the index of the book
    book_idx = book_to_idx[book]

    # Run the model to find neighbors
    distances, indices = model.kneighbors(book_user_mat.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)
    
    # The first neighbor is the book itself, so we skip it
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([idx_to_book[indices.flatten()[i]], distances.flatten()[i]])

    return [book, recommended_books]

In [ ]:
# Example test call from the instructions
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

In [ ]:
# Test function provided
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()